In [ ]:
import os
import torch
import pandas as pd
import scanpy as sc
import SpatialGlue


for path in ['../../datasets/Human_Lymph_Node_A1/','../../datasets/Human_Lymph_Node_D1/']:
    adata_omics1 = sc.read_h5ad(path + 'adata_RNA.h5ad')
    adata_omics2 = sc.read_h5ad(path + 'adata_ADT.h5ad')
    
    adata_omics1.var_names_make_unique()
    adata_omics2.var_names_make_unique()

    # Specify data type
    # Fix random seed
    from SpatialGlue.preprocess import fix_seed
    random_seed = 2022
    fix_seed(random_seed)

    from SpatialGlue.preprocess import clr_normalize_each_cell, pca


    # RNA
    sc.pp.filter_genes(adata_omics1, min_cells=10)
    sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
    sc.pp.normalize_total(adata_omics1, target_sum=1e4)
    sc.pp.log1p(adata_omics1)
    sc.pp.scale(adata_omics1)
    
    adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
    adata_omics1.obsm['feat'] = pca(adata_omics1_high, n_comps=30)
    
    # Protein
    adata_omics2 = clr_normalize_each_cell(adata_omics2)
    sc.pp.scale(adata_omics2)
    adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=30)

    from SpatialGlue.preprocess import construct_neighbor_graph
    data = construct_neighbor_graph(adata_omics1, adata_omics2,datatype='10x')

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    from SpatialGlue.SpatialGlue_pyG import Train_SpatialGlue
    #model = Train_SpatialGlue(data, device=device,datatype='10x')
    model = Train_SpatialGlue(data, device=device,datatype='Setreo-CITE-seq')
    
    # train model
    output = model.train()

    adata = adata_omics1.copy()
    adata.obsm["X_SpatialGlue"]=output['SpatialGlue'].copy()

    result_path=path.replace("datasets","results")
    adata.write_h5ad(result_path+'/adata_spatialglue.h5ad')

  0%|                                                                                            | 0/600 [00:00<?, ?it/s]/home/ws6tg/anaconda3/envs/scvi/lib/python3.9/site-packages/SpatialGlue/model.py:212: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.alpha = F.softmax(torch.squeeze(self.vu) + 1e-6)
100%|██████████████████████████████████████████████████████████████████████████████████| 600/600 [00:45<00:00, 13.06it/s]


Model training finished!

